In [1]:
import numpy as np
import torch 
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from io import open
import unicodedata
import string

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#all possible character tokens
all_letters = string.ascii_letters + " .,;:?!'-()\n"
all_letters_list = list(all_letters)
n_letters = len(all_letters) + 1

#hyperparameters
sequence_length = 40 #length of the sequence to learn from
offset = 1 #offset when drawing from the full text file, 1 means dense

next_length = 1 #length of sequence to predict
next_offset = 1 #how shifted the predicted sequence is forward

input_size = n_letters
hidden_size = 200
num_classes = n_letters
num_layers = 1
batch_size = 128
num_epochs = 150
learning_rate = 1e-3

cuda:0


In [3]:
#adapted from https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def readLines(filename):
    with open(filename, encoding='utf-8') as some_file:
        corpus = some_file.read()#.replace('\n', ' ')
        corpus = ''.join([i for i in corpus if not i.isdigit()])
        return corpus

def generate_sequences(filename, sequence_length=sequence_length, offset=offset, next_length=next_length):
    '''
    loads in the training data from a .txt file and preprocesses into 40-character chunks
    filename: name of the .txt file containing shakespearan sonnets
    length: the sequence_length used by the RNN
    offset: the offset between each sequence window, use offset=1 for the densent sequences
    '''
    corpus = readLines(filename)

    N = len(corpus)
    sequences = []
    next = []
    index = 0
    while index + sequence_length + 1 < N:
        sequences.append(corpus[index: index + sequence_length])
        next.append(corpus[index + sequence_length + next_offset -next_length: index + sequence_length + next_offset])
        index += offset

    return sequences, next

def inputTensor(sequences, length = sequence_length):
    '''
    Generates a tensor that contains one-hot matrices for charcter sequences.
    This is the correct format for the X inputs.
    '''
    tensor = torch.zeros(len(sequences), length, n_letters)
    for (i, seq) in enumerate(sequences):
        for li in range(length):
            letter = seq[li]
            tensor[i][li][all_letters.find(letter)] = 1
    return tensor

def categoryTensor(sequences, next_length=next_length):
    '''
    Generates a tensor that contains the token for each character.
    This is the correct format for the y labels.
    '''
    tokenized = torch.zeros(len(sequences), next_length)
    for (i, seq) in enumerate(sequences):
        for li in range(next_length):
            letter = seq[li]
        tokenized[i][li] = all_letters_list.index(letter)
    # tensor = torch.zeros(1, n_letters)
    # tensor[0][li] = 1
    # return tensor
    return tokenized.long()

In [4]:
#load and process the data for training
sequences, next = generate_sequences('data/shakespeare.txt')
X = inputTensor(sequences)
y = categoryTensor(next)

train_dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size, 
                                shuffle=True)

In [5]:
len(sequences)

97634

In [662]:
class RNN(nn.Module):
    '''
    Class for the RNN model in pytorch
    '''
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes) #next_length*
        #no softmax needed since it is calculated in the cross entropy
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        x, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        #select the length of the sequence that should be predicted
        x = x[:, -next_length:, :]
        #stack the sequence 
        out = x.reshape(x.size()[0]*x.size()[1], self.hidden_size)

        # Decode the hidden state of all of the timesteps
        #take all the length of the sequence you want to predict
        out = self.fc(out) 

        #resahpe the array so that it is in the format for cross entropy loss
        out = out.reshape(x.size()[0], x.size()[1], -1)
        out = torch.swapaxes(out, 1, 2)
        return out

In [663]:
def train():
    '''
    Trains a RNN given the hyparameters given above
    '''
    model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        cum_loss = 0
        for i, (X, y) in enumerate(train_loader):
            #X = X.reshape(-1, input_size, sequence_length).to(device)
            #X = X.reshape(-1, sequence_length, input_size).to(device)
            X = X.to(device)
            y = y.to(device)
            
            # Forward pass
            outputs = model(X)
            #print(outputs.shape)
            #print(y.shape)
            loss = criterion(outputs, y)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            cum_loss += loss
        
        if (epoch) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                    .format(epoch+1, num_epochs, cum_loss.item()))
    
    return model

def sample(model, temperature, start_seq=["shall i compare thee to a summer's day?\n"], max_length = 560):
    '''
    Generates a poem
    model: trained RNN model
    start_seq: a 40 character starting seed for the poem generation
    max_length: how many characters to generate
    '''

    with torch.no_grad():  # no need to track history in sampling
        input = inputTensor(start_seq)
        output_seq = start_seq[0]
        np.random.RandomState(42)

        for i in range(max_length):
            input = input.to(device)
            output = model(input)
            softmax = nn.Softmax(dim=1)
            
            #deterministic sampling, take the most likely letter
            if temperature == 0:
                topvs, topis = output.topk(1, dim = 1)
                #print(topis.shape)
                for i in range(next_offset):
                    topi = topis[0][0][next_length - 1 + i]
                    letter = all_letters[topi]   
                    output_seq += letter
            
            #probabilisitc sampling based on a temperature parameter  
            else:
                output /= temperature
                output = softmax(output)
                probs = (output[0,:,-1]).cpu()
                #for numerical stability
                probs /= (probs.sum()+1e-5)
                
                for i in range(next_offset):
                    indices = range(n_letters)
                    index = np.argmax(np.random.multinomial(1, probs))
                    #index = np.random.choice(indices, p=probs)
                    letter = all_letters_list[index]
                    output_seq += letter
        
            input = inputTensor([output_seq[-40:]])

        return output_seq

In [664]:
model = train()

Epoch [1/150], Loss: 2025.5798
Epoch [11/150], Loss: 1140.8790
Epoch [21/150], Loss: 922.6141
Epoch [31/150], Loss: 719.8034
Epoch [41/150], Loss: 569.3052
Epoch [51/150], Loss: 472.6846
Epoch [61/150], Loss: 409.8349
Epoch [71/150], Loss: 367.9156
Epoch [81/150], Loss: 334.7524
Epoch [91/150], Loss: 307.0707
Epoch [101/150], Loss: 279.2007
Epoch [111/150], Loss: 262.9477
Epoch [121/150], Loss: 248.2036
Epoch [131/150], Loss: 232.6705
Epoch [141/150], Loss: 231.0606


In [674]:
print(sample(model, temperature = 0))

shall i compare thee to a summer's day?
Thou art more loves to the very abrous fidle,
And yet not still even his black with mune.
  In thy sweet formering of their world expues,
For that say more that which fline in seem,
And say in his beauty shall to thee doth live,
  And since honour more than thou souldst in me,
Hine eyes, your lous'st me not so much rest
Onfeiticy, but by thy self with pure han eres.


                   
Then stoply fair that my time that case sone.
  To tiully me to make our name, she less
Is wond of mine on shamonous a many.
Naturay drien presage of thee despised,
But 


In [671]:
print(sample(model, temperature = 0.25))

shall i compare thee to a summer's day?
Thou art me, thou should thou wilt besmoun,
As those lipse that this shall gave my heart'st burt,
And so my plays where my love as your lies,
And so by these vainted acter is bend,
That for this thought goner, which is so greater,
But not then thou my love tame, her the sun
 outs do turn for my says our presict triff
With leathe her to thee requite upon brow?
Why of those would beauge we lack upon thethted,
Mine own truth voights which to ensuray peeds.
  You whom she is near suity to steep to be.
Af of hunot after that I in thy art?
Or cally of poor and


In [672]:
print(sample(model, temperature = 0.75))

shall i compare thee to a summer's day?
Thou art more bonds in their murity sight,
Ditied by hister-blace eachers wretch doth lack,
Whon of love that terges to every hade,
  So those that which thou restaling so vend.
If thou most love to whose rendos excelleghed:
A trave alter in all alone shall tone's past.
Letthenoud, bluck intermita inds dow:
  To sourkent thou the least, some false old,
And keeps thee I have still the store,
  But sweets of thee, 'Nat this auris affooted.


                   
I ne' from me for dowh hell with d-ain heece,
And by that I my body the contenced,
And but a fil


In [673]:
print(sample(model, temperature = 1.5))

shall i compare thee to a summer's day?
How have eye knoulgly detyoly chadly lien,
To play as for hore, why when that you should po most can,
He capenury cantel fronds be recods,
Or can uper canngry in other bold,
Thy objeess wat as your sweet as doth,
Unter theme leach unless to sleep, there graines.


                    
How makenemant, more int'ring with mancest,
The clear besteet forly sleak extile hey,
Goor better it were beremone) shall wond colfaded note.
  Orieviol, whoch welldingums devosed view,
Yet I this, manying no, and my love swear,
Thy should kind, but they with thy altaints s
